#1. 채점 함수의 이해
일단은 과제 설명에 앞서서, 앞으로 수행한 과제를 채점하는 함수에대한 설명부터 하겠습니다. <br> [여기](https://www.notion.so/240827-12d126fa69bd4d4cb1eb200007875842)에서 언급한 바와 같이 두 문장의 점수를 측정할 때, 어떤 방식으로 토크나이저가 문장을 분해하는 지가 ROUGE와 BLEU점수를 크게 좌우합니다.<br>
그래서 단어 단위로 분해, 형태소 단위로 분해 했을 때 각각의 점수를 한번에 표로 정리하는 함수를 만들었습니다.<br>
아래아래에 있는 코드블럭의 실행결과를 읽어보면 무슨 뜻인지 이해됩니다.

In [ ]:
# 구글 코랩에서 명령어 앞에 !를 치고 실행하면 vscode에서 터미널에 명령어를 치는 것과 같음
!pip install konlpy
!apt-get install -y openjdk-11-jdk
!pip install jpype1
!pip install pandas

In [ ]:
from konlpy.tag import Okt, Kkma
from collections import Counter
import math
import pandas as pd

# Okt와 Kkma 형태소 분석기 생성
okt = Okt()
kkma = Kkma()

# n-gram 생성 함수
def get_ngrams(tokens, n):
    return [tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

# ROUGE 계산 함수
def calculate_rouge(reference, candidate, n=1):
    ref_ngrams = get_ngrams(reference, n)
    cand_ngrams = get_ngrams(candidate, n)

    ref_counter = Counter(ref_ngrams)
    cand_counter = Counter(cand_ngrams)

    overlap = sum((ref_counter & cand_counter).values())

    precision = overlap / len(cand_ngrams) if len(cand_ngrams) > 0 else 0
    recall = overlap / len(ref_ngrams) if len(ref_ngrams) > 0 else 0
    f1_score = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0

    return precision, recall, f1_score

# BLEU 계산 함수
def calculate_bleu(reference, candidate, max_n=4):
    precisions = []
    for n in range(1, max_n+1):
        ref_ngrams = get_ngrams(reference, n)
        cand_ngrams = get_ngrams(candidate, n)

        ref_counter = Counter(ref_ngrams)
        cand_counter = Counter(cand_ngrams)

        overlap = sum((ref_counter & cand_counter).values())
        precision = overlap / len(cand_ngrams) if len(cand_ngrams) > 0 else 0
        precisions.append(precision)

    if all(p == 0 for p in precisions):
        bleu_score = 0
    else:
        bleu_score = math.exp(sum([math.log(p) if p > 0 else -999999 for p in precisions]) / max_n)

    ref_len = len(reference)
    cand_len = len(candidate)
    brevity_penalty = math.exp(1 - ref_len / cand_len) if cand_len < ref_len else 1

    bleu_score *= brevity_penalty

    return bleu_score

# ROUGE-L 계산 함수 (Longest Common Subsequence 기반)
def lcs(X, Y):
    m = len(X)
    n = len(Y)
    dp = [[0] * (n + 1) for i in range(m + 1)]

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if X[i - 1] == Y[j - 1]:
                dp[i][j] = dp[i - 1][j - 1] + 1
            else:
                dp[i][j] = max(dp[i - 1][j], dp[i][j - 1])

    return dp[m][n]

def calculate_rouge_l(reference, candidate):
    lcs_length = lcs(reference, candidate)
    precision = lcs_length / len(candidate) if len(candidate) > 0 else 0
    recall = lcs_length / len(reference) if len(reference) > 0 else 0
    f1_score = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0
    return precision, recall, f1_score

# 점수를 계산하고 표로 반환하는 함수
def calculate_scores(reference, candidate):
    # Okt를 사용한 토큰화
    reference_tokens_okt = okt.morphs(reference)
    candidate_tokens_okt = okt.morphs(candidate)

    # Kkma를 사용한 토큰화
    reference_tokens_kkma = kkma.morphs(reference)
    candidate_tokens_kkma = kkma.morphs(candidate)

    # Okt 기반 점수 계산
    bleu_okt = calculate_bleu(reference_tokens_okt, candidate_tokens_okt)
    rouge1_okt = calculate_rouge(reference_tokens_okt, candidate_tokens_okt, n=1)
    rouge2_okt = calculate_rouge(reference_tokens_okt, candidate_tokens_okt, n=2)
    rougeL_okt = calculate_rouge_l(reference_tokens_okt, candidate_tokens_okt)

    # Kkma 기반 점수 계산
    bleu_kkma = calculate_bleu(reference_tokens_kkma, candidate_tokens_kkma)
    rouge1_kkma = calculate_rouge(reference_tokens_kkma, candidate_tokens_kkma, n=1)
    rouge2_kkma = calculate_rouge(reference_tokens_kkma, candidate_tokens_kkma, n=2)
    rougeL_kkma = calculate_rouge_l(reference_tokens_kkma, candidate_tokens_kkma)

    # 결과를 DataFrame으로 정리
    scores_df = pd.DataFrame({
        'Metric': ['BLEU', 'ROUGE-1 Precision', 'ROUGE-1 Recall', 'ROUGE-1 F1',
                   'ROUGE-2 Precision', 'ROUGE-2 Recall', 'ROUGE-2 F1',
                   'ROUGE-L Precision', 'ROUGE-L Recall', 'ROUGE-L F1'],
        '단어': [bleu_okt, rouge1_okt[0], rouge1_okt[1], rouge1_okt[2],
                rouge2_okt[0], rouge2_okt[1], rouge2_okt[2],
                rougeL_okt[0], rougeL_okt[1], rougeL_okt[2]],
        '형태소': [bleu_kkma, rouge1_kkma[0], rouge1_kkma[1], rouge1_kkma[2],
                 rouge2_kkma[0], rouge2_kkma[1], rouge2_kkma[2],
                 rougeL_kkma[0], rougeL_kkma[1], rougeL_kkma[2]]
    })

    return scores_df

# 예시 문장
reference = "고양이가 나무 위로 올라갔다."
candidate = "나무 위로 고양이가 올라갔다."

# 점수 계산 및 결과 출력
print("<< 점수를 측정하려는 문장 >>")
print("reference:", reference)
print("candidate:", candidate)
print("\n<< 두 문장의 단어 단위 구분결과 >>")
print( okt.morphs(reference), okt.morphs(candidate))
print("\n<< 두 문장의 형태소 단위 구분결과 >>")
print(kkma.morphs(reference), kkma.morphs(candidate))
scores_df = calculate_scores(reference, candidate)
print("\n<< 두 문장의 단어/형태소 로 구분시 점수 >>")
print(scores_df)


#3. 본격적인 과제 설명
이번 과제를 통해 알아내야하는 것은 크게 세가지 입니다.

**1.   어떤 전처리 데이터가 프롬프트 엔지니어링 시 인공지능이 더 잘 이해하는가**<br>
현재는 똑같은 원본 데이터를 두가지 방식으로 전처리한 서로다른 두 종류의 데이터가 있습니다. 아직 어떤 데이터가 인공지능이 이해하기 편한지 알 수 없기에 이번 실습 과제에서 두 데이터를 모두 실험해보고 어떤 데이터가 전반적으로 점수가 더 잘나오는지 알려줘야합니다(노션에 정리해줘요). <br><br>
**첫번째 종류**는 [전체 표와 "highlighted_cells"항목이 존재하는 형태](https://raw.githubusercontent.com/beefed-up-geek/HCLT-KACL2024/main/Taeyoon_notebooks/full_example.txt)입니다.
> 데이터를 보면 전체 표가 들어가있고, highlighted_cells를 통해 전체 표의 어떤 부분지 중요한지 보여주고 있습니다. 이러한 데이터는 <U>인공지능이 표 전반에 대한 이해가 가능하다는 장점</U>이 있습니다. 하지만 <U>표가 너무 길어지면 highlighted_cells에 집중 할 수 없다는 단점</U>이 있습니다.

**두번째 종류**는 [전체 표에서 highlighted_cells과 그와 관련된 cell들만 추출한 형태](https://raw.githubusercontent.com/beefed-up-geek/HCLT-KACL2024/main/Taeyoon_notebooks/only_highloght_example.txt)입니다.
> 데이터를 보면 highlighted_cell항목이 없어진 것을 알 수 있습니다. 왜냐하면 어차피 highlighted_cells와 그와 관련된 항목들을 제외하고 모두 삭제했기 때문에 불필요하기 때문입니다. 이러한 형태는 <U>전체 표가 아무리 길어도 인공지능이 highlighted_cells만 볼 수 있기 때문에 집중을 훨씬 잘한다는 장점</U>이 있습니다. 하지만 반대로 <U>highlighted_cells만 볼 수 있기 때문에 표 전체에 대한 이해는 할 수 없다는 단점</U>이 있습니다.

두가지 데이터중 어떤 데이터가 더 적합할지 노션에 정리해서 알려주세요.<br><br>
**2.   표 유형별 프롬프트 엔지니어링** <br>
저희 데이터에는 7000개의 표와 각각에 대한 분석이 존재합니다. 이러한 데이터들을 분석해본 결과 인공지능이 수행해야하는 분석의 유형은 크게 3종류로 구분 가능했습니다.
1.  **정의 또는 설명** | 약 30~40% | [예시](https://raw.githubusercontent.com/beefed-up-geek/HCLT-KACL2024/main/Taeyoon_notebooks/def_example.txt)
2. **비교 대조** | 약 20~30% | [예시](https://raw.githubusercontent.com/beefed-up-geek/HCLT-KACL2024/main/Taeyoon_notebooks/comparison_example.txt)
3. **통계, 수치분석** | 약 25~40% | [예시](https://raw.githubusercontent.com/beefed-up-geek/HCLT-KACL2024/main/Taeyoon_notebooks/statistic_example.txt)

앞으로의 과제에서는 위 3가지 유형의 문제에 대하여 각각 프롬프트 엔지니어링 하게되고, 엔지니어링된 프롬프트의 성능을 평가하기위해 점수를 측정하게됩니다. <br><br>

**3. 최종 프롬프트 엔지니어링**<br>
마지막에서는 데이터의 유형과 관계없이 모든 유형의 데이터를 처리 가능한 프롬프트를 만들어줘야합니다. 이러한 프롬프트를 만든 후에, 인공지능이 이러한 프롬프트로 전체 데이터를 처리했을 때 몇점이 나오는지 평가합니다. <br>




#4. 과제 노트북 다운로드

오늘의 과제 노트북은 두개입니다. 1번에서 설명한 것과 같이 전처리한 데이터의 유형에 따라 노트북을 구분했습니다. 아래의 링크에서 노트북 파일을 다운받아주세요. (다운로드 버튼은 오른쪽 위에있음)


*   [전체 표와 "highlighted_cells"항목이 존재하는 형태](https://github.com/beefed-up-geek/HCLT-KACL2024/blob/main/Taeyoon_notebooks/240828_1.ipynb)
*   [전체 표에서 highlighted_cells과 그와 관련된 cell들만 추출한 형태](https://github.com/beefed-up-geek/HCLT-KACL2024/blob/main/Taeyoon_notebooks/240828_2.ipynb)

